In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(SimpleDense, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
            name="weight"
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True,
            name="bias"
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [3]:
dense_layer = SimpleDense(4, 2)

In [4]:
dense_layer.weights

[<tf.Variable 'weight:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.04301587, -0.02869543, -0.06771871,  0.12656045],
        [-0.0383564 ,  0.00517027,  0.07481531,  0.03325538]],
       dtype=float32)>,
 <tf.Variable 'bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [5]:
dense_layer.trainable_weights

[<tf.Variable 'weight:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.04301587, -0.02869543, -0.06771871,  0.12656045],
        [-0.0383564 ,  0.00517027,  0.07481531,  0.03325538]],
       dtype=float32)>,
 <tf.Variable 'bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [6]:
dense_layer.non_trainable_weights

[]

In [7]:
dense_layer.trainable

True

In [8]:
dense_layer.trainable = False

In [9]:
dense_layer.trainable_weights

[]

In [10]:
dense_layer.non_trainable_weights

[<tf.Variable 'weight:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.04301587, -0.02869543, -0.06771871,  0.12656045],
        [-0.0383564 ,  0.00517027,  0.07481531,  0.03325538]],
       dtype=float32)>,
 <tf.Variable 'bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

More Attributes

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#attributes

In [ ]:
x = tf.ones((2, 2))
Dense_layer = SimpleDense(4, 2)
y = dense_layer(x)
print(y)

tf.Tensor(
[[ 0.03571117 -0.12981753  0.0807825   0.06444939]
 [ 0.03571117 -0.12981753  0.0807825   0.06444939]], shape=(2, 4), dtype=float32)


By using `add_weight` method you can quickly create weigthts.

In [11]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(SimpleDense, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
Dense_layer = SimpleDense(4, 2)
y = dense_layer(x)
print(y)

tf.Tensor(
[[ 0.00465947 -0.02352516  0.0070966   0.15981583]
 [ 0.00465947 -0.02352516  0.0070966   0.15981583]], shape=(2, 4), dtype=float32)


In many cases, you may not know in advance the size of your inputs, and you would like to lazily create weights when that value becomes known, some time after instantiating the layer.

In the Keras API, we recommend creating layer weights in the `build(self, inputs_shape)` method of your layer. Like this:

In [12]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32):
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="zeros", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [13]:
dense_layer = SimpleDense(32)

In [14]:
dense_layer.weights

[]

The `call()` method of your layer will automatically run `build` the first time it is called. You now have a layer that's lazy and thus easier to use:

In [15]:
y = dense_layer(x)

In [16]:
dense_layer.weights

[<tf.Variable 'simple_dense_2/Variable:0' shape=(2, 32) dtype=float32, numpy=
 array([[ 0.0254224 ,  0.08004832,  0.06651141, -0.0639386 , -0.0155388 ,
          0.03223818, -0.0148812 ,  0.05661449,  0.07901189, -0.07481324,
         -0.01352238,  0.01820998,  0.0065708 ,  0.03833068, -0.01472153,
          0.0047923 , -0.09909044, -0.10059012,  0.09269513,  0.02396589,
          0.04490618,  0.11528528,  0.06080676,  0.01673501, -0.0485043 ,
         -0.04942962, -0.02539317, -0.12480115,  0.01121544,  0.03394528,
          0.07384913, -0.04822714],
        [-0.02683528,  0.02848235, -0.00935802,  0.05733574,  0.04399313,
         -0.0007192 , -0.1349039 , -0.01318345, -0.03292204, -0.03289424,
          0.01097736, -0.06509515,  0.06451034, -0.01510103, -0.00435992,
         -0.0148325 ,  0.03377907,  0.03644446, -0.00015354, -0.01762462,
         -0.03460966, -0.02442684,  0.00249608,  0.03802105, -0.07523502,
          0.01994103,  0.08183937,  0.01714282,  0.07077188,  0.01956559

If you assign a Layer instance as an attribute of another Layer, the outer layer will start tracking the weights created by the inner layer.

We recommend creating such sublayers in the `__init__()` method and leave it to the first `__call__()` to trigger building their weights.

In [17]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = tf.keras.layers.Dense(32)
        self.linear_2 = tf.keras.layers.Dense(32)
        self.linear_3 = SimpleDense(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6


Some layers, in particular the BatchNormalization layer and the Dropout layer, have different behaviors during training and inference. For such layers, it is standard practice to expose a training (boolean) argument in the `call()` method.

By exposing this argument in `call()`, you enable the built-in training and evaluation loops (e.g. `fit()`) to correctly use the layer in training and inference.

In [ ]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs